In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [ ]:
spark

In [3]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType
from pyspark.ml.pipeline import PipelineModel

In [4]:
globals()['models_loaded'] = False
globals()['my_model'] = None

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    # df.show()
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # globals()['my_model'] = PipelineModel.load('./lr1.model')
        globals()['my_model'] = PipelineModel.load('./lr2.model')
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model (uncomment below):
    
    df_result = globals()['my_model'].transform(df)
    df_result.drop("words", "cv", "features", "rawPrediction", "probability").show()

In [5]:
ssc = StreamingContext(sc, 10)

In [6]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [7]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [8]:
scc_t.stop()

NameError: name 'scc_t' is not defined

23/05/13 10:46:45 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:46:45 WARN BlockManager: Block input-0-1683967604800 replicated to only 0 peer(s) instead of 1 peers
23/05/13 10:46:50 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:46:50 WARN BlockManager: Block input-0-1683967609800 replicated to only 0 peer(s) instead of 1 peers


23/05/13 10:46:51 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:46:51 WARN BlockManager: Block input-0-1683967610800 replicated to only 0 peer(s) instead of 1 peers


========= 2023-05-13 10:46:50 =========


23/05/13 10:46:54 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:46:54 WARN BlockManager: Block input-0-1683967613800 replicated to only 0 peer(s) instead of 1 peers


...

========= 2023-05-13 10:49:00 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2369390|    1|138222699|I knew in the end...|
|2369390|    0|138222356|Got no flow to it...|
+-------+-----+---------+--------------------+

+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+
| app_id|label|review_id|         review_text|               words|                  cv|            features|       rawPrediction|probability|prediction|
+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+
|2369390|    1|138222699|I knew in the end...|[i, knew, in, the...|(13759,[0,2,5,12,...|(13759,[0,2,5,12,...|[402.402255702151...|  [1.0,0.0]|       0.0|
|2369390|    0|138222356|Got no flow to it...|[got, no, flow,

23/05/13 10:49:01 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:49:01 WARN BlockManager: Block input-0-1683967741200 replicated to only 0 peer(s) instead of 1 peers
23/05/13 10:49:02 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:49:02 WARN BlockManager: Block input-0-1683967742200 replicated to only 0 peer(s) instead of 1 peers
23/05/13 10:49:07 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:49:07 WARN BlockManager: Block input-0-1683967747200 replicated to only 0 peer(s) instead of 1 peers


========= 2023-05-13 10:49:10 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2369390|    0|138221738|I always got FPS ...|
|2369390|    0|138221590|cut scene after c...|
|2369390|    0|138221205|Game sucks. Not e...|
+-------+-----+---------+--------------------+

+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+
| app_id|label|review_id|         review_text|               words|                  cv|            features|       rawPrediction|probability|prediction|
+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+
|2369390|    0|138221738|I always got FPS ...|[i, always, got, ...|(13759,[0,1,2,5,8...|(13759,[0,1,2,5,8...|[-20092.903542510...|  [0.0,1.0]|       1.0|
|2369390|    0

23/05/13 10:49:12 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:49:12 WARN BlockManager: Block input-0-1683967752200 replicated to only 0 peer(s) instead of 1 peers
23/05/13 10:49:15 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:49:15 WARN BlockManager: Block input-0-1683967755200 replicated to only 0 peer(s) instead of 1 peers
23/05/13 10:49:19 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/13 10:49:19 WARN BlockManager: Block input-0-1683967759200 replicated to only 0 peer(s) instead of 1 peers


========= 2023-05-13 10:49:20 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|2369390|    1|138221003|Fun game. Terribl...|
|2349830|    1|138222191|its a fun game to...|
|1559600|    1|138221793|I played the demo...|
+-------+-----+---------+--------------------+

+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+
| app_id|label|review_id|         review_text|               words|                  cv|            features|       rawPrediction|probability|prediction|
+-------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------+
|2369390|    1|138221003|Fun game. Terribl...|[fun, game., terr...|(13759,[3,9,20,36...|(13759,[3,9,20,36...|[33249.4211590750...|  [1.0,0.0]|       0.0|
|2349830|    1